In [ ]:
# Download data from site -- http://ai.stanford.edu/~amaas/data/sentiment/
# p.234
#import pyprind # may need to do >sudo easy_install pip, then >pip install pyprind --user
import pandas as pd
import os

In [ ]:
pwd()

In [ ]:
'''
#pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg':0}

df = pd.DataFrame()

for s in ('test','train'):
    for l in ('pos','neg'):
        path = '/Users/jim/Desktop/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
#                try:
                    txt = infile.read()
                    df = df.append([ [txt, labels[l]] ], ignore_index=True)
#                    pbar.update()
#                except:
#                    pass

df.columns = ['review', 'sentiment']
'''

In [1]:
'''
# add try-except so that reading in data wont crash.
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('/Users/jim/Desktop/aclImdb/movie_data.csv', index = False, encoding='utf-8')
'''


SyntaxError: invalid syntax (<ipython-input-1-d3eab5af1b8a>, line 8)

In [2]:
#################
# Start here    #
#################
import pyprind
import pandas as pd
import os

df = pd.read_csv('/Users/jim/Desktop/aclImdb/movie_data.csv', encoding='utf-8')


ImportError: No module named 'pyprind'

In [ ]:
df.head(3)

In [ ]:
# Bag of Word model
# 1. create a vocabulary of unique tokens (or words)
# 2. construct a feature vector for each document, features store count
#    of words per document

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

docs = np.array(['The sun is shining', 
                 'The weather is sweet',
                 'The sun is shining and the weather is sweet'])

bag = count.fit_transform(docs)

print(count.vocabulary_)
print(bag.toarray())

In [ ]:
# tf(t,d) - raw term frequencies (t: term, d: nos times term t appears in doc d)
# tf-idf(t,d) - term frequency inverse document frequency
# tf-idf = tf(t,d) * idf(t,x)  = tf(t,d) * log( [1+nd]/[1+df(d,t)] ) 


In [ ]:
# TfidTransformer
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

In [ ]:
# so if the term "the" shows up lots of times, does that mean it's important?
# how can we make terms that shows up lots of times across documents, less important
# let's normalize by the times these terms show up across documents.

# employ : [nos of docs containing term "the" ]/[total nos of documents]

# if term appear often, give it less emphasis

# tf-idf(t,d) = tf(t,2)*(idf(t,d)+1)
# with idf(t,d) = log ([1+total nos of docs]/[1+nos of docs containing term t])


In [ ]:
import re
def preprocessor(text):
# find '<' then anything not '>' [^>], [^>]* 0 or more prefix, then close with '>'    
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) 
    # eyes[:,;,=], optional nose [-], and mouth[),(,D,P)]
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [ ]:
tmp = 'is seven.<br /><br />Title (Brazil): Not Available'

print(preprocessor(tmp))
print(preprocessor('</a>This :) is :( a test :-)!' ))
print(re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', '</a>This :) is :( a test :-)!'))

In [ ]:
df['review'] = df['review'].apply(preprocessor)

In [ ]:
df.shape

In [4]:
# p.242 Processing documents into tokens
# split the sentence/corpora into individual elements
def tokenizer(text):
    return text.split()

In [5]:
tokenizer('running like running and thus they run')

['running', 'like', 'running', 'and', 'thus', 'they', 'run']

In [6]:
# word stemming, tranforming word into their root form
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [7]:
tokenizer_porter('running like running and thus they run')


['run', 'like', 'run', 'and', 'thu', 'they', 'run']

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dawn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('english')  # stop words have little meaning eg. a, is, and, has, etc. 
[w for w in tokenizer_porter('a runner likes running and runs a lot') 
 if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [11]:
[w for w in tokenizer_porter(df['reviews']) if w not in stop]

AttributeError: 'WordListCorpusReader' object has no attribute 'worimport'

In [ ]:
# pg. 244
# Training a Logistic Regression model for document classification
# (X,y)
#X_train = df.loc[:25000, 'review'].values
#y_train = df.loc[:25000, 'sentiment'].values

#X_test  = df.loc[25000:, 'review'].values
#y_test  = df.loc[25000:, 'sentiment'].values

X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values

X_test  = df.loc[2500:5000, 'review'].values
y_test  = df.loc[2500:5000, 'sentiment'].values

print(y_test.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False)


In [ ]:
param_grid = [
              {'vect__ngram_range':[(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer], #, tokenizer_porter],
               'clf__penalty': ['l2'], # ['l1', 'l2'],
               'clf__C': [100]}, #1.0, 10.0, 100.0]},
              
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer], #, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C':[100] } #.0,10.0,100.0]}
                ]


In [ ]:
lr_tfidf = Pipeline([ ('vect', tfidf) ,
                      ('clf',  LogisticRegression(random_state=0))])



In [ ]:
gs_lr_tfidf = GridSearchCV( lr_tfidf, param_grid,
                          scoring = 'accuracy',
                          cv = 5, verbose = 1,
                          n_jobs = 1) # n_jobs -1 uses all computer cores

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

In [ ]:
print('The Best parameter set: %s' % gs_lr_tfidf.best_params_)


In [ ]:
print('CV Accuracy: %.3f'
     % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))


In [ ]:
# p.246
# Working with bigger data -- online algos and out-of-core learning

import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
def tokenizer(text):  # converts reviews to lower case, take out non-words, and put emoticons at the end
    text = re.sub('<[^>]*>', '', text) #
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8')  as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2]) 
            yield text, label 


In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y
        

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error = 'ignore',
                         n_features = 2**21,
                         preprocessor = None,
                         tokenizer = tokenizer)
clf = SGDClassifier(loss='log', random_state = 1, n_iter = 1)
doc_stream = stream_docs(path = '/Users/jim/Desktop/aclImdb/movie_data.csv')


In [ ]:
#import pyprind
classes = np.array([0, 1])
print(classes)
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes = classes) 


In [ ]:
X_test, y_test = get_minibatch(doc_stream, size = 500)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))



# Homework 4 - "mini-Kaggle competition"

In [ ]:
# HWK 4 - "In-class IMDB Kaggle competition" 

# 1. Use the smaller sample data per above for your train/test split 
#    (note you may use the whole data set too!)
#
#    X_train = df.loc[:2500, 'review'].values
#    y_train = df.loc[:2500, 'sentiment'].values
#    X_test  = df.loc[2500:5000, 'review'].values
#    y_test  = df.loc[2500:5000, 'sentiment'].values

# 2. Employ any technique learnt in class or outside of class to arrive at 
#    the best ML engine model for predicting movie sentiment base on reviews
#
#    Do 5-fold cross-validation and report your best model's train/test accuracy, f1-score, and auc
#    The winner will be the ML algo model that has the highest f1-score on both the training and test data.
#    Just as in the regular Kaggle competition combining models will be acceptable (collusion is ok!). 
#    However, please turn in your notebook individually to Blackbaord. Let the games begin!  
